In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
#@title Meto data del precio
def mini(col):
  if isinstance(col, str):
    return col
  else:
    v1='__'.join(col).strip()
    return f'birth/{v1}'
df=pd.read_pickle('/content/drive/MyDrive/Tesis Economia/Cambio climatico/agus/fullLondonV3.pkl')
#df.to_csv('fullLondonV3.csv')
df=df.drop(np.arange(2021,2051,1),level=1)
#trabajo con el sector de country of birth

df.columns=[mini(col) for col in df.columns.values]

#trabajo con el sector de los precios

y1=[f'Detached_Average_Price/{i}' for i in np.arange(1,13,1)]
y2=[f'Semi_Detached_Average_Price/{i}' for i in np.arange(1,13,1)]
y3=[f'Terraced_Average_Price/{i}' for i in np.arange(1,13,1)]
y4=[f'Flat_Average_Price/{i}' for i in np.arange(1,13,1)]
output1=df.loc[:,y1]
output2=df.loc[:,y2]
output3=df.loc[:,y3]
output4=df.loc[:,y4]

df['yMean1'] = output1.mean(axis=1)
df['yMean2'] = output2.mean(axis=1)
df['yMean3'] = output3.mean(axis=1)
df['yMean4'] = output4.mean(axis=1)

for i in np.arange(1,5,1):
  a='{col}'
  

  exec(f'output{i}=output{i}.apply(np.log,inplace=True)')
  exec(f"output{i}.columns = [f'ln{a}' for col in output{i}.columns.values]")
  exec(f"df['lnyMean{i}'] = output{i}.mean(axis=1)")

#trabajo con el sector de country of birth


df


interInflows  interOutflows  ...   lnyMean3   lnyMean4
Borough              Year                               ...                      
Barking and Dagenham 1995           NaN            NaN  ...  10.912681  10.691847
                     1996           NaN            NaN  ...  10.909939  10.691881
                     1997           NaN            NaN  ...  10.990044  10.740463
                     1998           NaN            NaN  ...  11.063541  10.804892
                     1999           NaN            NaN  ...  11.143287  10.891996
...                                 ...            ...  ...        ...        ...
Westminster          2016       13477.0         7497.0  ...  14.286381  13.752591
                     2017       19459.0         6779.0  ...  14.329452  13.822426
                     2018       17752.0         8873.0  ...  14.305731  13.776337
                     2019           NaN            NaN  ...  14.253770  13.707298
                     2020           NaN            NaN  ...  14.287804  13.712838

[858 rows x 489 columns]

In [ ]:
#simplific el dataset
data=df.copy()
#dropeo codes
data.drop('Code',axis=1,inplace=True)
data.drop('birth/Code____',axis=1,inplace=True)
data.drop('nationality/Code____',axis=1,inplace=True)

#inter flow solo dejo el net
#data.drop(['interInflows','interOutflows'],axis=1,inplace=True)

#dropeo las bandas de tax porque me parece que no te cobran muchos impuestos segun el valor de propiedad, no creo que genere distorsiones
data.drop(['Band A','Band B','Band C','Band D','Band E','Band F','Band G','Band H'],axis=1,inplace=True)

#cambio un cacho el country of birth
data.drop(['birth/European Union__European Union EU14__',
           'birth/European Union__European Union EU8__',
           'birth/European Union__European Union EU2__',
           'birth/European Union__European Union Other__',
           'birth/Non-European Union__Asia__Middle East and Central Asia',
           'birth/Non-European Union__Asia__East Asia',
           'birth/Non-European Union__Asia__South Asia',
           'birth/Non-European Union__Asia__South East Asia',
           'birth/European Union__European Union__',
           'birth/Non-European Union__All__',
           'birth/Non-European Union__All__',
           'birth/Non-European Union__Other Europe__',
           'birth/Non-European Union__Asia__All',
           'birth/Non-European Union__Rest of the World__All',
           'birth/Non-European Union__Rest of the World__Sub-Saharan Africa',
           'birth/Non-European Union__Rest of the World__North Africa',
           'birth/Non-European Union__Rest of the World__North America',
           'birth/Non-European Union__Rest of the World__Central and South America',
           'birth/Non-European Union__Rest of the World__Oceania'],axis=1,inplace=True)

data.drop('Source',axis=1,inplace=True)

#ahora con nacionalidad
data.drop(['nationality/European Union__European Union EU14__',
           'nationality/European Union__European Union EU8__',
           'nationality/European Union__European Union EU2__',
           'nationality/European Union__European Union Other__',
           'nationality/Non-European Union__Asia__Middle East and Central Asia',
           'nationality/Non-European Union__Asia__East Asia',
           'nationality/Non-European Union__Asia__South Asia',
           'nationality/Non-European Union__Asia__South East Asia',
           'nationality/European Union__European Union__',
           'nationality/Non-European Union__All__',
           'nationality/Non-European Union__All__',
           'nationality/Non-European Union__Other Europe__',
           'nationality/Non-European Union__Asia__All',
           'nationality/Non-European Union__Rest of the World__All',
           'nationality/Non-European Union__Rest of the World__Sub-Saharan Africa',
           'nationality/Non-European Union__Rest of the World__North Africa',
           'nationality/Non-European Union__Rest of the World__North America',
           'nationality/Non-European Union__Rest of the World__Central and South America',
           'nationality/Non-European Union__Rest of the World__Oceania'],axis=1,inplace=True)
#con respecto al crimen
data.drop(['Criminal Damage','Drugs','Fraud or Forgery','Other Notifiable Offences','Robbery','Sexual Offences','Theft and Handling','Violence Against the Person'],axis=1,inplace=True)
#tiro sobre hectareas
data.drop(['Population','Population_per_hectare','Square_Kilometres'],axis=1,inplace=True)
#religion tiro los numeros absolutos
data.drop(['Numbers__Christian','Numbers__Buddhist','Numbers__Hindu','Numbers__Jewish','Numbers__Muslim','Numbers__Sikh','Numbers__Any Other Religion','Numbers__No Religion','Numbers__Total'],axis=1,inplace=True)

data

In [3]:
#@title Meto la data de la inundacion
#meto la data sobre la inundacion
def numerizeYear(df):
  df.reset_index(inplace=True)
  df['Year']=pd.to_numeric(df['Year'])
  df.set_index(['Borough','Year'],inplace=True)
  return df
def boroughSetter(df):
  df.reset_index(inplace=True)
  df['Borough'] = df['Borough'].str.replace(' Of ', ' of ')
  df['Borough'] = df['Borough'].str.replace(' Upon ', ' upon ')
  df['Borough'] = df['Borough'].str.replace(' & ', ' and ')
  df['Borough'] = df['Borough'].str.replace('City of Westminster', 'Westminster')
  
  df.set_index(['Borough'],inplace=True)
  return df

def junter(full,new):
  new=numerizeYear(new)
  new=boroughSetter(new)
  result=pd.concat([full, new],axis=1)
  return result

def checker(df,start,finish):
  print('la lenght es ',len(df))
  for i in np.arange(start,finish+1,1):
    print(len(df.loc[(slice(None),i),:]))
  return None
water=pd.read_excel('/content/drive/MyDrive/Tesis Economia/Cambio climatico/excel/inunuk/inunUK.xlsx')
#water.to_csv('inunUK.csv')
water=boroughSetter(water)
water.drop('index',inplace=True,axis=1)
water.sort_index(inplace=True)
df0=df.copy()
df0=df0.reset_index().set_index(['Year','Borough']).sort_index()

#pd.concat([df0,water],join='inner',axis=1)

df0 = water.join(df0, how="inner")
df0=df0.reset_index().set_index(['Borough','Year']).sort_index()
#df0.to_csv('fullLondonV4.csv')
#df0.to_pickle('fullLondonV4.pkl')


In [ ]:
#@title aca estan las columnas viejas
#creo las variables para trabajar con partes del dataframe
v1=['Inun', '0-2%', '2%-5%', ' 5%-15%', '15%-30%', '30%-50%','50%-70%', '70%-85%', '85%-95%', '95%-98%', '98%-100%']
v2=['interInflows', 'interOutflows', 'interNetflows']
v3=['Band A','Band B', 'Band C', 'Band D', 'Band E', 'Band F', 'Band G','Band H']
v4=['birth/Total____', 
    'birth/United Kingdom____',
    'birth/Non-United Kingdom____',
    'birth/European Union__European Union__',
    'birth/European Union__European Union EU14__',
    'birth/European Union__European Union EU8__',
    'birth/European Union__European Union EU2__',
    'birth/European Union__European Union Other__',
    'birth/Non-European Union__All__',
    'birth/Non-European Union__Other Europe__',
    'birth/Non-European Union__Asia__All',
    'birth/Non-European Union__Asia__Middle East and Central Asia',
    'birth/Non-European Union__Asia__East Asia',
    'birth/Non-European Union__Asia__South Asia',
    'birth/Non-European Union__Asia__South East Asia',
    'birth/Non-European Union__Rest of the World__All',
    'birth/Non-European Union__Rest of the World__Sub-Saharan Africa',
    'birth/Non-European Union__Rest of the World__North Africa',
    'birth/Non-European Union__Rest of the World__North America',
    'birth/Non-European Union__Rest of the World__Central and South America',
    'birth/Non-European Union__Rest of the World__Oceania']
v5=[' Total electors', ' Attainers']
v6=['All recorded offences',
    'Burglary', 'Criminal Damage', 'Drugs', 'Fraud or Forgery',
    'Mid-year estimates', 'Other Notifiable Offences', 'Robbery',
    'Sexual Offences', 'Theft and Handling',
    'Violence Against the Person']
v7=['Population',
       'Inland_Area _Hectares', 'Total_Area_Hectares',
       'Population_per_hectare', 'Square_Kilometres',
       'Population_per_square_kilometre']
v8=['nationality/Total____',
       'nationality/British____', 'nationality/Non-British____',
       'nationality/European Union__European Union__',
       'nationality/European Union__European Union EU14__',
       'nationality/European Union__European Union EU8__',
       'nationality/European Union__European Union EU2__',
       'nationality/European Union__European Union Other__',
       'nationality/Non-European Union__All__',
       'nationality/Non-European Union__Other Europe__',
       'nationality/Non-European Union__Asia__All',
       'nationality/Non-European Union__Asia__Middle East and Central Asia',
       'nationality/Non-European Union__Asia__East Asia',
       'nationality/Non-European Union__Asia__South Asia',
       'nationality/Non-European Union__Asia__South East Asia',
       'nationality/Non-European Union__Rest of the World__All',
       'nationality/Non-European Union__Rest of the World__Sub-Saharan Africa',
       'nationality/Non-European Union__Rest of the World__North Africa',
       'nationality/Non-European Union__Rest of the World__North America',
       'nationality/Non-European Union__Rest of the World__Central and South America',
       'nationality/Non-European Union__Rest of the World__Oceania']
v9=['Decisions made during the year per 1,000 households',
       'Households accommodated by the authority on 31 March per 1,000 households']
v10=['Numbers__Christian', 'Numbers__Buddhist', 'Numbers__Hindu',
       'Numbers__Jewish', 'Numbers__Muslim', 'Numbers__Sikh',
       'Numbers__Any Other Religion', 'Numbers__No Religion',
       'Numbers__Total', 'Percentages__Christian',
       'Percentages__Buddhist', 'Percentages__Hindu',
       'Percentages__Jewish', 'Percentages__Muslim', 'Percentages__Sikh',
       'Percentages__Any Other Religion', 'Percentages__No Religion']

v11=['flow__Barking and Dagenham/Age 0-3',
       'flow__Barking and Dagenham/Age 11-15',
       'flow__Barking and Dagenham/Age 4-10', 'flow__Barnet/Age 0-3',
       'flow__Barnet/Age 11-15', 'flow__Barnet/Age 4-10',
       'flow__Bexley/Age 0-3', 'flow__Bexley/Age 11-15',
       'flow__Bexley/Age 4-10', 'flow__Brent/Age 0-3',
       'flow__Brent/Age 11-15', 'flow__Brent/Age 4-10',
       'flow__Bromley/Age 0-3', 'flow__Bromley/Age 11-15',
       'flow__Bromley/Age 4-10', 'flow__Camden/Age 0-3',
       'flow__Camden/Age 11-15', 'flow__Camden/Age 4-10',
       'flow__City of London/Age 0-3', 'flow__City of London/Age 11-15',
       'flow__City of London/Age 4-10', 'flow__Croydon/Age 0-3',
       'flow__Croydon/Age 11-15', 'flow__Croydon/Age 4-10',
       'flow__Ealing/Age 0-3', 'flow__Ealing/Age 11-15',
       'flow__Ealing/Age 4-10', 'flow__Enfield/Age 0-3',
       'flow__Enfield/Age 11-15', 'flow__Enfield/Age 4-10',
       'flow__Greenwich/Age 0-3', 'flow__Greenwich/Age 11-15',
       'flow__Greenwich/Age 4-10', 'flow__Hackney/Age 0-3',
       'flow__Hackney/Age 11-15', 'flow__Hackney/Age 4-10',
       'flow__Hammersmith and Fulham/Age 0-3',
       'flow__Hammersmith and Fulham/Age 11-15',
       'flow__Hammersmith and Fulham/Age 4-10', 'flow__Haringey/Age 0-3',
       'flow__Haringey/Age 11-15', 'flow__Haringey/Age 4-10',
       'flow__Harrow/Age 0-3', 'flow__Harrow/Age 11-15',
       'flow__Harrow/Age 4-10', 'flow__Havering/Age 0-3',
       'flow__Havering/Age 11-15', 'flow__Havering/Age 4-10',
       'flow__Hillingdon/Age 0-3', 'flow__Hillingdon/Age 11-15',
       'flow__Hillingdon/Age 4-10', 'flow__Hounslow/Age 0-3',
       'flow__Hounslow/Age 11-15', 'flow__Hounslow/Age 4-10',
       'flow__Islington/Age 0-3', 'flow__Islington/Age 11-15',
       'flow__Islington/Age 4-10', 'flow__Kensington and Chelsea/Age 0-3',
       'flow__Kensington and Chelsea/Age 11-15',
       'flow__Kensington and Chelsea/Age 4-10',
       'flow__Kingston upon Thames/Age 0-3',
       'flow__Kingston upon Thames/Age 11-15',
       'flow__Kingston upon Thames/Age 4-10', 'flow__Lambeth/Age 0-3',
       'flow__Lambeth/Age 11-15', 'flow__Lambeth/Age 4-10',
       'flow__Lewisham/Age 0-3', 'flow__Lewisham/Age 11-15',
       'flow__Lewisham/Age 4-10', 'flow__Merton/Age 0-3',
       'flow__Merton/Age 11-15', 'flow__Merton/Age 4-10',
       'flow__Newham/Age 0-3', 'flow__Newham/Age 11-15',
       'flow__Newham/Age 4-10', 'flow__Redbridge/Age 0-3',
       'flow__Redbridge/Age 11-15', 'flow__Redbridge/Age 4-10',
       'flow__Richmond upon Thames/Age 0-3',
       'flow__Richmond upon Thames/Age 11-15',
       'flow__Richmond upon Thames/Age 4-10', 'flow__Southwark/Age 0-3',
       'flow__Southwark/Age 11-15', 'flow__Southwark/Age 4-10',
       'flow__Sutton/Age 0-3', 'flow__Sutton/Age 11-15',
       'flow__Sutton/Age 4-10', 'flow__Tower Hamlets/Age 0-3',
       'flow__Tower Hamlets/Age 11-15', 'flow__Tower Hamlets/Age 4-10',
       'flow__Waltham Forest/Age 0-3', 'flow__Waltham Forest/Age 11-15',
       'flow__Waltham Forest/Age 4-10', 'flow__Wandsworth/Age 0-3',
       'flow__Wandsworth/Age 11-15', 'flow__Wandsworth/Age 4-10',
       'flow__Westminster/Age 0-3', 'flow__Westminster/Age 11-15',
       'flow__Westminster/Age 4-10']
v12=['age/0', 'age/1', 'age/2', 'age/3',
       'age/4', 'age/5', 'age/6', 'age/7', 'age/8', 'age/9', 'age/10',
       'age/11', 'age/12', 'age/13', 'age/14', 'age/15', 'age/16',
       'age/17', 'age/18', 'age/19', 'age/20', 'age/21', 'age/22',
       'age/23', 'age/24', 'age/25', 'age/26', 'age/27', 'age/28',
       'age/29', 'age/30', 'age/31', 'age/32', 'age/33', 'age/34',
       'age/35', 'age/36', 'age/37', 'age/38', 'age/39', 'age/40',
       'age/41', 'age/42', 'age/43', 'age/44', 'age/45', 'age/46',
       'age/47', 'age/48', 'age/49', 'age/50', 'age/51', 'age/52',
       'age/53', 'age/54', 'age/55', 'age/56', 'age/57', 'age/58',
       'age/59', 'age/60', 'age/61', 'age/62', 'age/63', 'age/64',
       'age/65', 'age/66', 'age/67', 'age/68', 'age/69', 'age/70',
       'age/71', 'age/72', 'age/73', 'age/74', 'age/75', 'age/76',
       'age/77', 'age/78', 'age/79', 'age/80', 'age/81', 'age/82',
       'age/83', 'age/84', 'age/85', 'age/86', 'age/87', 'age/88',
       'age/89', 'age/ ALL AGES', 'age/90+']
v13=['Detached_Average_Price/1',
       'Detached_Average_Price/2', 'Detached_Average_Price/3',
       'Detached_Average_Price/4', 'Detached_Average_Price/5',
       'Detached_Average_Price/6', 'Detached_Average_Price/7',
       'Detached_Average_Price/8', 'Detached_Average_Price/9',
       'Detached_Average_Price/10', 'Detached_Average_Price/11',
       'Detached_Average_Price/12', 'Detached_Index/1',
       'Detached_Index/2', 'Detached_Index/3', 'Detached_Index/4',
       'Detached_Index/5', 'Detached_Index/6', 'Detached_Index/7',
       'Detached_Index/8', 'Detached_Index/9', 'Detached_Index/10',
       'Detached_Index/11', 'Detached_Index/12',
       'Detached_Monthly_Change/1', 'Detached_Monthly_Change/2',
       'Detached_Monthly_Change/3', 'Detached_Monthly_Change/4',
       'Detached_Monthly_Change/5', 'Detached_Monthly_Change/6',
       'Detached_Monthly_Change/7', 'Detached_Monthly_Change/8',
       'Detached_Monthly_Change/9', 'Detached_Monthly_Change/10',
       'Detached_Monthly_Change/11', 'Detached_Monthly_Change/12',
       'Detached_Annual_Change/1', 'Detached_Annual_Change/2',
       'Detached_Annual_Change/3', 'Detached_Annual_Change/4',
       'Detached_Annual_Change/5', 'Detached_Annual_Change/6',
       'Detached_Annual_Change/7', 'Detached_Annual_Change/8',
       'Detached_Annual_Change/9', 'Detached_Annual_Change/10',
       'Detached_Annual_Change/11', 'Detached_Annual_Change/12',
       'Semi_Detached_Average_Price/1', 'Semi_Detached_Average_Price/2',
       'Semi_Detached_Average_Price/3', 'Semi_Detached_Average_Price/4',
       'Semi_Detached_Average_Price/5', 'Semi_Detached_Average_Price/6',
       'Semi_Detached_Average_Price/7', 'Semi_Detached_Average_Price/8',
       'Semi_Detached_Average_Price/9', 'Semi_Detached_Average_Price/10',
       'Semi_Detached_Average_Price/11', 'Semi_Detached_Average_Price/12',
       'Semi_Detached_Index/1', 'Semi_Detached_Index/2',
       'Semi_Detached_Index/3', 'Semi_Detached_Index/4',
       'Semi_Detached_Index/5', 'Semi_Detached_Index/6',
       'Semi_Detached_Index/7', 'Semi_Detached_Index/8',
       'Semi_Detached_Index/9', 'Semi_Detached_Index/10',
       'Semi_Detached_Index/11', 'Semi_Detached_Index/12',
       'Semi_Detached_Monthly_Change/1', 'Semi_Detached_Monthly_Change/2',
       'Semi_Detached_Monthly_Change/3', 'Semi_Detached_Monthly_Change/4',
       'Semi_Detached_Monthly_Change/5', 'Semi_Detached_Monthly_Change/6',
       'Semi_Detached_Monthly_Change/7', 'Semi_Detached_Monthly_Change/8',
       'Semi_Detached_Monthly_Change/9',
       'Semi_Detached_Monthly_Change/10',
       'Semi_Detached_Monthly_Change/11',
       'Semi_Detached_Monthly_Change/12', 'Semi_Detached_Annual_Change/1',
       'Semi_Detached_Annual_Change/2', 'Semi_Detached_Annual_Change/3',
       'Semi_Detached_Annual_Change/4', 'Semi_Detached_Annual_Change/5',
       'Semi_Detached_Annual_Change/6', 'Semi_Detached_Annual_Change/7',
       'Semi_Detached_Annual_Change/8', 'Semi_Detached_Annual_Change/9',
       'Semi_Detached_Annual_Change/10', 'Semi_Detached_Annual_Change/11',
       'Semi_Detached_Annual_Change/12', 'Terraced_Average_Price/1',
       'Terraced_Average_Price/2', 'Terraced_Average_Price/3',
       'Terraced_Average_Price/4', 'Terraced_Average_Price/5',
       'Terraced_Average_Price/6', 'Terraced_Average_Price/7',
       'Terraced_Average_Price/8', 'Terraced_Average_Price/9',
       'Terraced_Average_Price/10', 'Terraced_Average_Price/11',
       'Terraced_Average_Price/12', 'Terraced_Index/1',
       'Terraced_Index/2', 'Terraced_Index/3', 'Terraced_Index/4',
       'Terraced_Index/5', 'Terraced_Index/6', 'Terraced_Index/7',
       'Terraced_Index/8', 'Terraced_Index/9', 'Terraced_Index/10',
       'Terraced_Index/11', 'Terraced_Index/12',
       'Terraced_Monthly_Change/1', 'Terraced_Monthly_Change/2',
       'Terraced_Monthly_Change/3', 'Terraced_Monthly_Change/4',
       'Terraced_Monthly_Change/5', 'Terraced_Monthly_Change/6',
       'Terraced_Monthly_Change/7', 'Terraced_Monthly_Change/8',
       'Terraced_Monthly_Change/9', 'Terraced_Monthly_Change/10',
       'Terraced_Monthly_Change/11', 'Terraced_Monthly_Change/12',
       'Terraced_Annual_Change/1', 'Terraced_Annual_Change/2',
       'Terraced_Annual_Change/3', 'Terraced_Annual_Change/4',
       'Terraced_Annual_Change/5', 'Terraced_Annual_Change/6',
       'Terraced_Annual_Change/7', 'Terraced_Annual_Change/8',
       'Terraced_Annual_Change/9', 'Terraced_Annual_Change/10',
       'Terraced_Annual_Change/11', 'Terraced_Annual_Change/12',
       'Flat_Average_Price/1', 'Flat_Average_Price/2',
       'Flat_Average_Price/3', 'Flat_Average_Price/4',
       'Flat_Average_Price/5', 'Flat_Average_Price/6',
       'Flat_Average_Price/7', 'Flat_Average_Price/8',
       'Flat_Average_Price/9', 'Flat_Average_Price/10',
       'Flat_Average_Price/11', 'Flat_Average_Price/12', 'Flat_Index/1',
       'Flat_Index/2', 'Flat_Index/3', 'Flat_Index/4', 'Flat_Index/5',
       'Flat_Index/6', 'Flat_Index/7', 'Flat_Index/8', 'Flat_Index/9',
       'Flat_Index/10', 'Flat_Index/11', 'Flat_Index/12',
       'Flat_Monthly_Change/1', 'Flat_Monthly_Change/2',
       'Flat_Monthly_Change/3', 'Flat_Monthly_Change/4',
       'Flat_Monthly_Change/5', 'Flat_Monthly_Change/6',
       'Flat_Monthly_Change/7', 'Flat_Monthly_Change/8',
       'Flat_Monthly_Change/9', 'Flat_Monthly_Change/10',
       'Flat_Monthly_Change/11', 'Flat_Monthly_Change/12',
       'Flat_Annual_Change/1', 'Flat_Annual_Change/2',
       'Flat_Annual_Change/3', 'Flat_Annual_Change/4',
       'Flat_Annual_Change/5', 'Flat_Annual_Change/6',
       'Flat_Annual_Change/7', 'Flat_Annual_Change/8',
       'Flat_Annual_Change/9', 'Flat_Annual_Change/10',
       'Flat_Annual_Change/11', 'Flat_Annual_Change/12']
v14=['yMean1',
       'yMean2', 'yMean3', 'yMean4', 'lnyMean1', 'lnyMean2', 'lnyMean3',
       'lnyMean4']


In [ ]:
#@title cambio el dataframe

#trabajo con el dataframe

#populations ['birth/Total____','Population','nationality/Total____','Numbers__Total','age/ ALL AGES']
#despues de sacar todos los porcentajes, tiro las otras estimaciones y dejo 'Population'

#dropeo codes
data=df.copy()
data.drop(['Code',
         'birth/Code____',
         'nationality/Code____',
         'Source'],axis=1,inplace=True)

#creo porcentajes
for i in v4:
  data[f'{i}/percent']=data[i]/data['birth/Total____']
data.drop('birth/Total____/percent',axis=1,inplace=True)
for i in v8:
  data[f'{i}/percent']=data[i]/data['nationality/Total____']
data.drop('nationality/Total____/percent',axis=1,inplace=True)
data.drop('nationality/Total____',axis=1,inplace=True)
data.drop('birth/Total____',axis=1,inplace=True)

#trabajo un poco con las hectareas ['Inland_Area _Hectares', 'Total_Area_Hectares','Population_per_hectare']
data.drop(['Square_Kilometres','Population_per_square_kilometre'],axis=1,inplace=True)
data['noLand']=data['Total_Area_Hectares']-data['Inland_Area _Hectares']
data.drop('Total_Area_Hectares',axis=1,inplace=True)

#trabajo con election roll
data['percent_attainers']=data[' Attainers']/data[' Total electors']

#burglar, hay que sacar porcentajes
for i in v6:
  data[f'{i}/percent']=data[i]/data['All recorded offences']
data.drop('All recorded offences/percent',axis=1,inplace=True)
data.drop('Mid-year estimates/percent',axis=1,inplace=True)

#tengo que trabajar con 90+, puedo hacer un promedio, donde 91 es 90+ /10
for i in np.arange(90,101,1):
  data[f'age/{i}']=data['age/90+']/10

data.drop('age/90+',axis=1,inplace=True)

#en price, que significa average price, index, change?
#es interesante intentar simplificar los flows, pero por ahora remmplazo los nan por 0
data.loc[(slice(None),np.arange(2002,2017,1)),v11]=data.loc[(slice(None),np.arange(2002,2017,1)),v11].fillna(0)

#dropeo la poblacion
data.drop(['Numbers__Total','age/ ALL AGES'],axis=1,inplace=True)



#data.to_csv('fullLondonV7.csv')
#data.to_pickle('fullLondonV7.pkl')


In [ ]:
#@title listas con nombres de las variables
w=['Inun', '0-2%', '2%-5%', ' 5%-15%', '15%-30%', '30%-50%',
       '50%-70%', '70%-85%', '85%-95%', '95%-98%', '98%-100%']
x1=['interInflows', 'interOutflows', 'interNetflows']
x2=['Band A',
       'Band B', 'Band C', 'Band D', 'Band E', 'Band F', 'Band G',
       'Band H']
x3=['birth/United Kingdom____',
       'birth/Non-United Kingdom____',
       'birth/European Union__European Union__',
       'birth/European Union__European Union EU14__',
       'birth/European Union__European Union EU8__',
       'birth/European Union__European Union EU2__',
       'birth/European Union__European Union Other__',
       'birth/Non-European Union__All__',
       'birth/Non-European Union__Other Europe__',
       'birth/Non-European Union__Asia__All',
       'birth/Non-European Union__Asia__Middle East and Central Asia',
       'birth/Non-European Union__Asia__East Asia',
       'birth/Non-European Union__Asia__South Asia',
       'birth/Non-European Union__Asia__South East Asia',
       'birth/Non-European Union__Rest of the World__All',
       'birth/Non-European Union__Rest of the World__Sub-Saharan Africa',
       'birth/Non-European Union__Rest of the World__North Africa',
       'birth/Non-European Union__Rest of the World__North America',
       'birth/Non-European Union__Rest of the World__Central and South America',
       'birth/Non-European Union__Rest of the World__Oceania']
x12=['birth/United Kingdom____/percent',
       'birth/Non-United Kingdom____/percent',
       'birth/European Union__European Union__/percent',
       'birth/European Union__European Union EU14__/percent',
       'birth/European Union__European Union EU8__/percent',
       'birth/European Union__European Union EU2__/percent',
       'birth/European Union__European Union Other__/percent',
       'birth/Non-European Union__All__/percent',
       'birth/Non-European Union__Other Europe__/percent',
       'birth/Non-European Union__Asia__All/percent',
       'birth/Non-European Union__Asia__Middle East and Central Asia/percent',
       'birth/Non-European Union__Asia__East Asia/percent',
       'birth/Non-European Union__Asia__South Asia/percent',
       'birth/Non-European Union__Asia__South East Asia/percent',
       'birth/Non-European Union__Rest of the World__All/percent',
       'birth/Non-European Union__Rest of the World__Sub-Saharan Africa/percent',
       'birth/Non-European Union__Rest of the World__North Africa/percent',
       'birth/Non-European Union__Rest of the World__North America/percent',
       'birth/Non-European Union__Rest of the World__Central and South America/percent',
       'birth/Non-European Union__Rest of the World__Oceania/percent']
x4=[' Total electors', ' Attainers','percent_attainers']
x5=['All recorded offences',
       'Burglary', 'Criminal Damage', 'Drugs', 'Fraud or Forgery',
       'Mid-year estimates', 'Other Notifiable Offences', 'Robbery',
       'Sexual Offences', 'Theft and Handling',
       'Violence Against the Person']
x14=['Burglary/percent',
       'Criminal Damage/percent', 'Drugs/percent',
       'Fraud or Forgery/percent', 'Other Notifiable Offences/percent',
       'Robbery/percent', 'Sexual Offences/percent',
       'Theft and Handling/percent',
       'Violence Against the Person/percent']
x6=['Population',
       'Inland_Area _Hectares', 'Population_per_hectare','noLand']
x7=['nationality/British____', 'nationality/Non-British____',
       'nationality/European Union__European Union__',
       'nationality/European Union__European Union EU14__',
       'nationality/European Union__European Union EU8__',
       'nationality/European Union__European Union EU2__',
       'nationality/European Union__European Union Other__',
       'nationality/Non-European Union__All__',
       'nationality/Non-European Union__Other Europe__',
       'nationality/Non-European Union__Asia__All',
       'nationality/Non-European Union__Asia__Middle East and Central Asia',
       'nationality/Non-European Union__Asia__East Asia',
       'nationality/Non-European Union__Asia__South Asia',
       'nationality/Non-European Union__Asia__South East Asia',
       'nationality/Non-European Union__Rest of the World__All',
       'nationality/Non-European Union__Rest of the World__Sub-Saharan Africa',
       'nationality/Non-European Union__Rest of the World__North Africa',
       'nationality/Non-European Union__Rest of the World__North America',
       'nationality/Non-European Union__Rest of the World__Central and South America',
       'nationality/Non-European Union__Rest of the World__Oceania']
x13=['nationality/British____/percent',
       'nationality/Non-British____/percent',
       'nationality/European Union__European Union__/percent',
       'nationality/European Union__European Union EU14__/percent',
       'nationality/European Union__European Union EU8__/percent',
       'nationality/European Union__European Union EU2__/percent',
       'nationality/European Union__European Union Other__/percent',
       'nationality/Non-European Union__All__/percent',
       'nationality/Non-European Union__Other Europe__/percent',
       'nationality/Non-European Union__Asia__All/percent',
       'nationality/Non-European Union__Asia__Middle East and Central Asia/percent',
       'nationality/Non-European Union__Asia__East Asia/percent',
       'nationality/Non-European Union__Asia__South Asia/percent',
       'nationality/Non-European Union__Asia__South East Asia/percent',
       'nationality/Non-European Union__Rest of the World__All/percent',
       'nationality/Non-European Union__Rest of the World__Sub-Saharan Africa/percent',
       'nationality/Non-European Union__Rest of the World__North Africa/percent',
       'nationality/Non-European Union__Rest of the World__North America/percent',
       'nationality/Non-European Union__Rest of the World__Central and South America/percent',
       'nationality/Non-European Union__Rest of the World__Oceania/percent']
x8=['Decisions made during the year per 1,000 households',
       'Households accommodated by the authority on 31 March per 1,000 households']
x9=['Numbers__Christian', 'Numbers__Buddhist', 'Numbers__Hindu',
       'Numbers__Jewish', 'Numbers__Muslim', 'Numbers__Sikh',
       'Numbers__Any Other Religion', 'Numbers__No Religion',
       'Percentages__Christian', 'Percentages__Buddhist',
       'Percentages__Hindu', 'Percentages__Jewish', 'Percentages__Muslim',
       'Percentages__Sikh', 'Percentages__Any Other Religion',
       'Percentages__No Religion']

x10=['flow__Barking and Dagenham/Age 0-3',
       'flow__Barking and Dagenham/Age 11-15',
       'flow__Barking and Dagenham/Age 4-10', 'flow__Barnet/Age 0-3',
       'flow__Barnet/Age 11-15', 'flow__Barnet/Age 4-10',
       'flow__Bexley/Age 0-3', 'flow__Bexley/Age 11-15',
       'flow__Bexley/Age 4-10', 'flow__Brent/Age 0-3',
       'flow__Brent/Age 11-15', 'flow__Brent/Age 4-10',
       'flow__Bromley/Age 0-3', 'flow__Bromley/Age 11-15',
       'flow__Bromley/Age 4-10', 'flow__Camden/Age 0-3',
       'flow__Camden/Age 11-15', 'flow__Camden/Age 4-10',
       'flow__City of London/Age 0-3', 'flow__City of London/Age 11-15',
       'flow__City of London/Age 4-10', 'flow__Croydon/Age 0-3',
       'flow__Croydon/Age 11-15', 'flow__Croydon/Age 4-10',
       'flow__Ealing/Age 0-3', 'flow__Ealing/Age 11-15',
       'flow__Ealing/Age 4-10', 'flow__Enfield/Age 0-3',
       'flow__Enfield/Age 11-15', 'flow__Enfield/Age 4-10',
       'flow__Greenwich/Age 0-3', 'flow__Greenwich/Age 11-15',
       'flow__Greenwich/Age 4-10', 'flow__Hackney/Age 0-3',
       'flow__Hackney/Age 11-15', 'flow__Hackney/Age 4-10',
       'flow__Hammersmith and Fulham/Age 0-3',
       'flow__Hammersmith and Fulham/Age 11-15',
       'flow__Hammersmith and Fulham/Age 4-10', 'flow__Haringey/Age 0-3',
       'flow__Haringey/Age 11-15', 'flow__Haringey/Age 4-10',
       'flow__Harrow/Age 0-3', 'flow__Harrow/Age 11-15',
       'flow__Harrow/Age 4-10', 'flow__Havering/Age 0-3',
       'flow__Havering/Age 11-15', 'flow__Havering/Age 4-10',
       'flow__Hillingdon/Age 0-3', 'flow__Hillingdon/Age 11-15',
       'flow__Hillingdon/Age 4-10', 'flow__Hounslow/Age 0-3',
       'flow__Hounslow/Age 11-15', 'flow__Hounslow/Age 4-10',
       'flow__Islington/Age 0-3', 'flow__Islington/Age 11-15',
       'flow__Islington/Age 4-10', 'flow__Kensington and Chelsea/Age 0-3',
       'flow__Kensington and Chelsea/Age 11-15',
       'flow__Kensington and Chelsea/Age 4-10',
       'flow__Kingston upon Thames/Age 0-3',
       'flow__Kingston upon Thames/Age 11-15',
       'flow__Kingston upon Thames/Age 4-10', 'flow__Lambeth/Age 0-3',
       'flow__Lambeth/Age 11-15', 'flow__Lambeth/Age 4-10',
       'flow__Lewisham/Age 0-3', 'flow__Lewisham/Age 11-15',
       'flow__Lewisham/Age 4-10', 'flow__Merton/Age 0-3',
       'flow__Merton/Age 11-15', 'flow__Merton/Age 4-10',
       'flow__Newham/Age 0-3', 'flow__Newham/Age 11-15',
       'flow__Newham/Age 4-10', 'flow__Redbridge/Age 0-3',
       'flow__Redbridge/Age 11-15', 'flow__Redbridge/Age 4-10',
       'flow__Richmond upon Thames/Age 0-3',
       'flow__Richmond upon Thames/Age 11-15',
       'flow__Richmond upon Thames/Age 4-10', 'flow__Southwark/Age 0-3',
       'flow__Southwark/Age 11-15', 'flow__Southwark/Age 4-10',
       'flow__Sutton/Age 0-3', 'flow__Sutton/Age 11-15',
       'flow__Sutton/Age 4-10', 'flow__Tower Hamlets/Age 0-3',
       'flow__Tower Hamlets/Age 11-15', 'flow__Tower Hamlets/Age 4-10',
       'flow__Waltham Forest/Age 0-3', 'flow__Waltham Forest/Age 11-15',
       'flow__Waltham Forest/Age 4-10', 'flow__Wandsworth/Age 0-3',
       'flow__Wandsworth/Age 11-15', 'flow__Wandsworth/Age 4-10',
       'flow__Westminster/Age 0-3', 'flow__Westminster/Age 11-15',
       'flow__Westminster/Age 4-10']

x11=['age/0', 'age/1', 'age/2', 'age/3',
       'age/4', 'age/5', 'age/6', 'age/7', 'age/8', 'age/9', 'age/10',
       'age/11', 'age/12', 'age/13', 'age/14', 'age/15', 'age/16',
       'age/17', 'age/18', 'age/19', 'age/20', 'age/21', 'age/22',
       'age/23', 'age/24', 'age/25', 'age/26', 'age/27', 'age/28',
       'age/29', 'age/30', 'age/31', 'age/32', 'age/33', 'age/34',
       'age/35', 'age/36', 'age/37', 'age/38', 'age/39', 'age/40',
       'age/41', 'age/42', 'age/43', 'age/44', 'age/45', 'age/46',
       'age/47', 'age/48', 'age/49', 'age/50', 'age/51', 'age/52',
       'age/53', 'age/54', 'age/55', 'age/56', 'age/57', 'age/58',
       'age/59', 'age/60', 'age/61', 'age/62', 'age/63', 'age/64',
       'age/65', 'age/66', 'age/67', 'age/68', 'age/69', 'age/70',
       'age/71', 'age/72', 'age/73', 'age/74', 'age/75', 'age/76',
       'age/77', 'age/78', 'age/79', 'age/80', 'age/81', 'age/82',
       'age/83', 'age/84', 'age/85', 'age/86', 'age/87', 'age/88',
       'age/89','age/90', 'age/91',
       'age/92', 'age/93', 'age/94', 'age/95', 'age/96', 'age/97',
       'age/98', 'age/99', 'age/100']
y1=['Detached_Average_Price/1', 'Detached_Average_Price/2',
       'Detached_Average_Price/3', 'Detached_Average_Price/4',
       'Detached_Average_Price/5', 'Detached_Average_Price/6',
       'Detached_Average_Price/7', 'Detached_Average_Price/8',
       'Detached_Average_Price/9', 'Detached_Average_Price/10',
       'Detached_Average_Price/11', 'Detached_Average_Price/12',
       'Detached_Index/1', 'Detached_Index/2', 'Detached_Index/3',
       'Detached_Index/4', 'Detached_Index/5', 'Detached_Index/6',
       'Detached_Index/7', 'Detached_Index/8', 'Detached_Index/9',
       'Detached_Index/10', 'Detached_Index/11', 'Detached_Index/12',
       'Detached_Monthly_Change/1', 'Detached_Monthly_Change/2',
       'Detached_Monthly_Change/3', 'Detached_Monthly_Change/4',
       'Detached_Monthly_Change/5', 'Detached_Monthly_Change/6',
       'Detached_Monthly_Change/7', 'Detached_Monthly_Change/8',
       'Detached_Monthly_Change/9', 'Detached_Monthly_Change/10',
       'Detached_Monthly_Change/11', 'Detached_Monthly_Change/12',
       'Detached_Annual_Change/1', 'Detached_Annual_Change/2',
       'Detached_Annual_Change/3', 'Detached_Annual_Change/4',
       'Detached_Annual_Change/5', 'Detached_Annual_Change/6',
       'Detached_Annual_Change/7', 'Detached_Annual_Change/8',
       'Detached_Annual_Change/9', 'Detached_Annual_Change/10',
       'Detached_Annual_Change/11', 'Detached_Annual_Change/12',
       'Semi_Detached_Average_Price/1', 'Semi_Detached_Average_Price/2',
       'Semi_Detached_Average_Price/3', 'Semi_Detached_Average_Price/4',
       'Semi_Detached_Average_Price/5', 'Semi_Detached_Average_Price/6',
       'Semi_Detached_Average_Price/7', 'Semi_Detached_Average_Price/8',
       'Semi_Detached_Average_Price/9', 'Semi_Detached_Average_Price/10',
       'Semi_Detached_Average_Price/11', 'Semi_Detached_Average_Price/12',
       'Semi_Detached_Index/1', 'Semi_Detached_Index/2',
       'Semi_Detached_Index/3', 'Semi_Detached_Index/4',
       'Semi_Detached_Index/5', 'Semi_Detached_Index/6',
       'Semi_Detached_Index/7', 'Semi_Detached_Index/8',
       'Semi_Detached_Index/9', 'Semi_Detached_Index/10',
       'Semi_Detached_Index/11', 'Semi_Detached_Index/12',
       'Semi_Detached_Monthly_Change/1', 'Semi_Detached_Monthly_Change/2',
       'Semi_Detached_Monthly_Change/3', 'Semi_Detached_Monthly_Change/4',
       'Semi_Detached_Monthly_Change/5', 'Semi_Detached_Monthly_Change/6',
       'Semi_Detached_Monthly_Change/7', 'Semi_Detached_Monthly_Change/8',
       'Semi_Detached_Monthly_Change/9',
       'Semi_Detached_Monthly_Change/10',
       'Semi_Detached_Monthly_Change/11',
       'Semi_Detached_Monthly_Change/12', 'Semi_Detached_Annual_Change/1',
       'Semi_Detached_Annual_Change/2', 'Semi_Detached_Annual_Change/3',
       'Semi_Detached_Annual_Change/4', 'Semi_Detached_Annual_Change/5',
       'Semi_Detached_Annual_Change/6', 'Semi_Detached_Annual_Change/7',
       'Semi_Detached_Annual_Change/8', 'Semi_Detached_Annual_Change/9',
       'Semi_Detached_Annual_Change/10', 'Semi_Detached_Annual_Change/11',
       'Semi_Detached_Annual_Change/12', 'Terraced_Average_Price/1',
       'Terraced_Average_Price/2', 'Terraced_Average_Price/3',
       'Terraced_Average_Price/4', 'Terraced_Average_Price/5',
       'Terraced_Average_Price/6', 'Terraced_Average_Price/7',
       'Terraced_Average_Price/8', 'Terraced_Average_Price/9',
       'Terraced_Average_Price/10', 'Terraced_Average_Price/11',
       'Terraced_Average_Price/12', 'Terraced_Index/1',
       'Terraced_Index/2', 'Terraced_Index/3', 'Terraced_Index/4',
       'Terraced_Index/5', 'Terraced_Index/6', 'Terraced_Index/7',
       'Terraced_Index/8', 'Terraced_Index/9', 'Terraced_Index/10',
       'Terraced_Index/11', 'Terraced_Index/12',
       'Terraced_Monthly_Change/1', 'Terraced_Monthly_Change/2',
       'Terraced_Monthly_Change/3', 'Terraced_Monthly_Change/4',
       'Terraced_Monthly_Change/5', 'Terraced_Monthly_Change/6',
       'Terraced_Monthly_Change/7', 'Terraced_Monthly_Change/8',
       'Terraced_Monthly_Change/9', 'Terraced_Monthly_Change/10',
       'Terraced_Monthly_Change/11', 'Terraced_Monthly_Change/12',
       'Terraced_Annual_Change/1', 'Terraced_Annual_Change/2',
       'Terraced_Annual_Change/3', 'Terraced_Annual_Change/4',
       'Terraced_Annual_Change/5', 'Terraced_Annual_Change/6',
       'Terraced_Annual_Change/7', 'Terraced_Annual_Change/8',
       'Terraced_Annual_Change/9', 'Terraced_Annual_Change/10',
       'Terraced_Annual_Change/11', 'Terraced_Annual_Change/12',
       'Flat_Average_Price/1', 'Flat_Average_Price/2',
       'Flat_Average_Price/3', 'Flat_Average_Price/4',
       'Flat_Average_Price/5', 'Flat_Average_Price/6',
       'Flat_Average_Price/7', 'Flat_Average_Price/8',
       'Flat_Average_Price/9', 'Flat_Average_Price/10',
       'Flat_Average_Price/11', 'Flat_Average_Price/12', 'Flat_Index/1',
       'Flat_Index/2', 'Flat_Index/3', 'Flat_Index/4', 'Flat_Index/5',
       'Flat_Index/6', 'Flat_Index/7', 'Flat_Index/8', 'Flat_Index/9',
       'Flat_Index/10', 'Flat_Index/11', 'Flat_Index/12',
       'Flat_Monthly_Change/1', 'Flat_Monthly_Change/2',
       'Flat_Monthly_Change/3', 'Flat_Monthly_Change/4',
       'Flat_Monthly_Change/5', 'Flat_Monthly_Change/6',
       'Flat_Monthly_Change/7', 'Flat_Monthly_Change/8',
       'Flat_Monthly_Change/9', 'Flat_Monthly_Change/10',
       'Flat_Monthly_Change/11', 'Flat_Monthly_Change/12',
       'Flat_Annual_Change/1', 'Flat_Annual_Change/2',
       'Flat_Annual_Change/3', 'Flat_Annual_Change/4',
       'Flat_Annual_Change/5', 'Flat_Annual_Change/6',
       'Flat_Annual_Change/7', 'Flat_Annual_Change/8',
       'Flat_Annual_Change/9', 'Flat_Annual_Change/10',
       'Flat_Annual_Change/11', 'Flat_Annual_Change/12']
y2=['yMean1',
       'yMean2', 'yMean3', 'yMean4']

y3=['lnyMean1', 'lnyMean2', 'lnyMean3',
       'lnyMean4']



In [4]:
#importo la data
df=pd.read_pickle('/content/drive/MyDrive/Tesis Economia/Cambio climatico/agus/fullLondonV7.pkl')
#df.to_csv('fullLondonV7.csv')
df

Inun  0-2%  2%-5%  ...  age/98  age/99  age/100
Borough              Year                     ...                         
Barking and Dagenham 1995     1     0      0  ...     NaN     NaN      NaN
                     1996     1     0      0  ...     NaN     NaN      NaN
                     1997     1     0      0  ...     NaN     NaN      NaN
                     1998     1     0      0  ...     NaN     NaN      NaN
                     1999     1     0      0  ...     NaN     NaN      NaN
...                         ...   ...    ...  ...     ...     ...      ...
Westminster          2016     1     0      0  ...     NaN     NaN      NaN
                     2017     1     0      0  ...     NaN     NaN      NaN
                     2018     1     0      0  ...     NaN     NaN      NaN
                     2019     1     0      0  ...     NaN     NaN      NaN
                     2020     1     0      0  ...     NaN     NaN      NaN

[858 rows x 547 columns]

In [ ]:
def graph(across,x,y):
  if across=='Borough':
  
  
    px.line(df.reset_index(),x='Year',y=y,color='Borough').show()
  else:


  
  
    px.scatter(df.reset_index(),x=x,y=y,color='Year').show()

#some graphs, los outputs se llaman lnyMean1	lnyMean2	lnyMean3	lnyMean4


#mainlnyMean3
graph('Year','Population_per_hectare','lnyMean2')
